In [ ]:
# The focus of this project is otimize crypto portfolios
# the result will be a table 

In [ ]:
# Imports
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# to manage the data from the api
import json 

# To get the data from the api
import requests

# to dataframes the math operations
import pandas as pd
import numpy as np

# to create the model
import tensorflow as tf

# to create graphs
import matplotlib.pyplot as plt

In [ ]:
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
tf.compat.v1.disable_eager_execution()

In [ ]:
# Use:
# pip install (name of the library)
# in case of trouble with the previous lines 

In [ ]:
# NumPy
np.__version__

In [ ]:
# Pandas
pd.__version__

In [ ]:
# TensorFlow
tf.__version__

In [ ]:
import matplotlib as m
m.__version__

In [ ]:
# Variables

In [ ]:
# List of cryptocurrencies that will compound my portfolio
cryptocurrencies = ["BTC", "ETH"]

In [ ]:
# Number of past days used to get data from the API
days_hist = 300

In [ ]:
# Dictionary to save the data collect from the API
crypto_hist = {}

In [ ]:
# Start the variable days_hist
hist_length = 0

In [ ]:
# medium return
return_m = {}

In [ ]:
# Dictionary to return the accumulated return
return_ac = {}

In [ ]:
# Function to collect data from the API

In [ ]:
# Function to generate index(column time) for the history transactions
def index_hist(hist):
    hist = hist.set_index('time')
    hist.index = pd.to_datetime(hist.index, unit = 's')
    return hist

In [ ]:
def get_hist(coin):
    endpoint_url = "https://min-api.cryptocompare.com/data/v2/histoday?fsym={}&tsym=USD&limit={:d}&extraParams=XXX".format(coin, days_hist)
    print(endpoint_url)
    res = requests.get(endpoint_url)
    hist = pd.DataFrame(json.loads(res.content)['Data']['Data']) # conversion to dataframe
    hist = index_hist(hist)
    return hist

In [ ]:
# Loop
def get_all():
    for cryptocoin in cryptocurrencies:
        crypto_hist[cryptocoin] = get_hist(cryptocoin)

In [ ]:
# Call the function get_all
get_all()

In [ ]:
# Define the length 
hist_length = len(crypto_hist[cryptocurrencies[0]])

In [ ]:
# Print the data
crypto_hist

In [ ]:
# Bitcoin head(5 first lines) history of the past 300 days
crypto_hist['BTC'].head()

### Return 

In [ ]:
# Function to calculate the return
def calc_return():
    for cryptocoin in cryptocurrencies:

        # Collect data
        hist = crypto_hist[cryptocoin]

        # Calculate the return
        hist['return'] = (hist['close'] - hist['open']) / hist['open']

        # Mean return
        media = hist["return"].mean()
        
        #return_medio[cryptocoin] = media

        # Accumulated return
        return_accumulated[cryptocoin] = (hist['return'] + 1).prod() - 1

        # Extra return 
        hist['return_extra'] = hist['return'] - media

        # Dataframe
        crypto_hist[cryptocoin] = hist

In [ ]:
calc_return()

In [ ]:
print(return_accumulated)

In [ ]:
# Dataset with the return columns
crypto_hist['BTC'].head()

In [ ]:
return_extra_matrix = np.zeros((hist_length, len(cryptocoins)))

In [ ]:
return_extra_matrix

In [ ]:
# Fullfilling the matrix above with the extra return 
for i in range(0, hist_length):
    for idx, cryptocoin in enumerate(cryptocoins):
        return_extra_matrix[i][idx] = crypto_hist[cryptocoin].iloc[i]['return_extra']

In [ ]:
# Visualize the matrix
return_extra_matrix_form = pd.DataFrame(return_extra_matrix, columns = cryptocoins)
return_extra_matrix_form.index = crypto_hist[cryptocoins[0]].index

return_extra_matrix_form

In [ ]:
prod_matrix = np.matmul(return_extra_matrix.transpose(), return_extra_matrix)
var_covar_matrix = prod_matrix / hist_length

In [ ]:
var_covar_matrix_form= pd.DataFrame(var_covar_matrix, columns = cryptocoins, index = cryptocoins)
var_covar_matrix_form

In [ ]:
# standard deviation
deviation_s = np.zeros((len(cryptocoins), 1))

for idx, cryptocoin in enumerate(cryptocoins):
    deviation_s[idx][0] = np.std(crypto_hist[cryptocoin]['return'])

In [ ]:
deviation_s_form = pd.DataFrame(deviation_s, columns = ['deviation_s'], index = cryptocoins)
deviation_s_form

In [ ]:
# deviation_s matrix
deviation_s_prod_matrix = np.matmul(deviation_s, deviation_s.transpose())

In [ ]:
deviation_s_prod_matrix_form = pd.DataFrame(deviation_s_prod_matrix, columns = cryptocoins, index = cryptocoins)
deviation_s_prod_matrix_form

In [ ]:
# Correlation matrix
matrix_cor = var_covar_matrix / deviation_s_prod_matrix

In [ ]:
matrix_cor_form = pd.DataFrame(matrix_cor, columns = cryptocoins, index = cryptocoins)
matrix_cor_form

### Model

Model:

Sqrt(Transpose (Wt.SD) * Correlation Matrix * Wt.SD)

In [ ]:
def minimize_volatility():

    # Start variables with TensorFlow
    coin_weights = tf.Variable(np.full((len(cryptocoins), 1), 1.0 / len(cryptocoins)))

    weighted_std_devs = tf.multiply(coin_weights, desvio_padrao)

    product_1 = tf.transpose(a=weighted_std_devs)

    product_2 = tf.matmul(product_1, matriz_cor)

    portfolio_variance = tf.matmul(product_2, weighted_std_devs)

    portfolio_volatility = tf.sqrt(tf.reduce_sum(input_tensor=portfolio_variance))

    # Model: Portfolio Volatility = Sqrt(Transpose (Wt.SD) * Correlation Matrix * Wt.SD)

    # Model hiperparametres
    learn_rate = 0.01
    steps = 10000

    # Iniciate the variables
    init = tf.compat.v1.global_variables_initializer()

    # Training using Gradient Descent to minimize the variance (volatility)
    train_step = tf.compat.v1.train.GradientDescentOptimizer(learn_rate).minimize(portfolio_volatility)

    # Session TF
    with tf.compat.v1.Session() as sess:
        sess.run(init)
        for i in range(steps):
            sess.run(train_step)
            if i % 1000 == 0 :
                print("[Iteration {:d}]".format(i))
                print("Weights", coin_weights.eval())
                print("Volatility: {:.2f}%".format(portfolio_volatility.eval() * 100))
                print("")

        return coin_weights.eval()

In [ ]:
result = minimize_volatility()

In [ ]:
print(result)

In [ ]:
# Print the result
result_form = pd.DataFrame(result * 100, index = cryptocoins, columns = ["% Volatility for the portfolio"])
result_form

### Models Constraints

In [ ]:
# Add limits
# Goal: minimize the variance
# Add constraints

def minimize_volatility_v2():

    coin_weights = tf.Variable(np.full((len(criptomoedas), 1), 1.0 / len(criptomoedas)))
    weighted_std_devs = tf.multiply(coin_weights, desvio_padrao)

    product_1 = tf.transpose(a=weighted_std_devs)
    product_2 = tf.matmul(product_1, matriz_cor)

    portfolio_variance = tf.matmul(product_2, weighted_std_devs)
    portfolio_volatility = tf.sqrt(tf.reduce_sum(input_tensor=portfolio_variance))

    # Constraints: sum([0..1, 0..1, ...]) = 1

    lower_than_zero = tf.greater( np.float64(0), coin_weights )
    zero_minimum_op = coin_weights.assign( tf.compat.v1.where (lower_than_zero, tf.zeros_like(coin_weights), coin_weights) )

    greater_than_one = tf.greater( coin_weights, np.float64(1) )
    unity_max_op = coin_weights.assign( tf.compat.v1.where (greater_than_one, tf.ones_like(coin_weights), coin_weights) )

    result_sum = tf.reduce_sum(input_tensor=coin_weights)
    unity_sum_op = coin_weights.assign(tf.divide(coin_weights, result_sum))

    constraints_op = tf.group(zero_minimum_op, unity_max_op, unity_sum_op)

    # Hiperparametres
    learning_rate = 0.01
    steps = 10000

    init = tf.compat.v1.global_variables_initializer()

    # Training the model using Gradient Descent to minimize the variance(volatility)
    optimize_op = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(portfolio_volatility)

    with tf.compat.v1.Session() as sess:
        sess.run(init)
        for i in range(steps):
            sess.run(optimize_op)
            sess.run(constraints_op)
            if i % 2500 == 0 :
                print("[Iteration {:d}]".format(i))
                print("Weights", coin_weights.eval())
                print("Volatility: {:.2f}%".format(portfolio_volatility.eval() * 100))
                print("")

        sess.run(constraints_op)
        return coin_weights.eval()

In [ ]:
# Training the model
result = minimize_volatility_v2()

In [ ]:
# Result
result_form = pd.DataFrame(result * 100, index = cryptocoins, columns = ["% Volatility to the Portfolio"])
result_form

### Part 3

In [ ]:
# Otimize weights to maximize the return/risc

import time
start = time.time()

def maximize_sharpe_ratio():

    # Variance

    coin_weights = tf.Variable(tf.random.uniform((len(cryptocoins), 1), dtype=tf.float64))
    weighted_std_devs = tf.multiply(coin_weights, standard_deviation)

    product_1 = tf.transpose(a=weighted_std_devs)
    product_2 = tf.matmul(product_1, matriz_cor)

    portfolio_variance = tf.matmul(product_2, weighted_std_devs)
    portfolio_volatility = tf.sqrt(tf.reduce_sum(input_tensor=portfolio_variance))


    # Return

    returns = np.full((len(cryptocoins), 1), 0.0)
    for coin_idx in range(0, len(cryptocoins)):
        returns[coin_idx] = return_ac[cryptocoins[coin_idx]]

    portfolio_return = tf.reduce_sum(input_tensor=tf.multiply(coin_weights, returns))


    # Return / Risc (Sharpe Ratio)
    # https://www.investopedia.com/terms/s/sharperatio.asp

    sharpe_ratio = tf.divide(portfolio_return, portfolio_volatility)


    # Constraints

    weights_sum = tf.reduce_sum(input_tensor=coin_weights)
    constraints_op = coin_weights.assign(tf.divide(tf.abs(coin_weights), tf.abs(weights_sum) ))


    # Hiperparametres
    learning_rate = 0.0001
    learning_rate = 0.0015
    steps = 10000

    # Training using Gradient Descent to minimize the variance (volatility)

    optimize_op = tf.compat.v1.train.GradientDescentOptimizer(learning_rate, use_locking = True).minimize(tf.negative(sharpe_ratio))

    init = tf.compat.v1.global_variables_initializer()

    with tf.compat.v1.Session() as sess:
        ratios = np.zeros(steps)
        returns = np.zeros(steps)
        sess.run(init)
        for i in range(steps):
            sess.run(optimize_op)
            sess.run(constraints_op)
            ratios[i] = sess.run(sharpe_ratio)
            returns[i] = sess.run(portfolio_return) * 100
            if i % 2000 == 0 :
                sess.run(constraints_op)
                print("[Iteration {:d}]".format(i))
                print("Volatility {:.2f} %".format(sess.run(portfolio_volatility)))
                print("Return {:.2f} %".format(sess.run(portfolio_return)*100))
                print("sharpe_ratio", sess.run(sharpe_ratio))
                print("")

        sess.run(constraints_op)
        print("Volatility {:.2f} %".format(sess.run(portfolio_volatility)))
        print("Return {:.2f} %".format(sess.run(portfolio_return)*100))
        print("sharpe_ratio", sess.run(sharpe_ratio))
        return sess.run(coin_weights)

weights = maximize_sharpe_ratio()

print("\n")
print("Time used {:f}s to conclude".format(time.time() - start))
pretty_weights = pd.DataFrame(weights * 100, index = cryptocoins, columns = ["% of the portfolio"])

pretty_weights